<a href="https://colab.research.google.com/github/sallumandya1995/grammar_coorection/blob/main/1-dec-2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
# https://www.youtube.com/watch?v=scJsty_DR3o
# https://www.numpyninja.com/post/question-answering-through-bert-in-10-steps
# https://huggingface.co/clips/mfaq
# https://www.youtube.com/watch?v=OlhNZg4gOvA

# https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline

# https://colab.research.google.com/github/kstathou/vector_engine/blob/master/notebooks/001_vector_search.ipynb#scrollTo=aecWwyrutyef
# https://www.youtube.com/watch?v=w1dMEWm7jBc

In [114]:
#to watch

# https://www.youtube.com/watch?v=ZxR38An5TQE


# https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb


In [115]:
#dhiraj 

# Semantic search with FAISS (TensorFlow)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [125]:
# comments_df.to_csv("output.csv")


In [2]:
import pandas as pd

import pyarrow as pa
import datasets

import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
comments_df = df[['title', 'field_solutiondescription','Key-Words','nid']]
comments_df['nid'] = comments_df['nid'].astype(str)
comments_df.head()

 






<ipython-input-2-9e2c92d580e2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_df['nid'] = comments_df['nid'].astype(str)


,title,field_solutiondescription,Key-Words,nid
0,$finance,sector financial servicesdata category financi...,"{'sector': 'financial servicesdata', 'category...",6533
1,House_price_prediction_final,house_price_prediction_final,['house_price_prediction_final'],6532
2,Diabetes_Patients,sector lifescience healthcaredata category hea...,"{'sector': 'lifescience healthcaredata', 'cate...",6531
3,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6530
4,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6529


In [3]:
comments_df.dropna(inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [4]:
comments_df.isnull().sum()

title                        0
field_solutiondescription    0
Key-Words                    0
nid                          0
dtype: int64

In [5]:
table = pa.Table.from_pandas(comments_df)

# create a datasets.arrow_dataset.Dataset object from the table
comments_dataset = datasets.arrow_dataset.Dataset(table)

In [6]:
def concatenate_text(examples):
    return {
        "text": examples["nid"]
        + " \n "
        + examples["title"]
        + " \n "
        + examples["field_solutiondescription"]
    }


comments_dataset = comments_dataset.map(concatenate_text)

  0%|          | 0/3245 [00:00<?, ?ex/s]

In [7]:
comments_dataset

Dataset({
    features: ['title', 'field_solutiondescription', 'Key-Words', 'nid', '__index_level_0__', 'text'],
    num_rows: 3245
})

In [8]:
comments_dataset

Dataset({
    features: ['title', 'field_solutiondescription', 'Key-Words', 'nid', '__index_level_0__', 'text'],
    num_rows: 3245
})

In [9]:
comments_dataset
import pandas as pd

# write the dataset to a CSV file
comments_dataset.to_csv('comments.csv')

# read the CSV file into a pandas data frame
df = pd.read_csv('comments.csv')

# view the first few records in the data frame
df.head()


Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

,Unnamed: 0,title,field_solutiondescription,Key-Words,nid,__index_level_0__,text
0,0,$finance,sector financial servicesdata category financi...,"{'sector': 'financial servicesdata', 'category...",6533,0,6533 \n $finance \n sector financial servicesd...
1,1,House_price_prediction_final,house_price_prediction_final,['house_price_prediction_final'],6532,1,6532 \n House_price_prediction_final \n house_...
2,2,Diabetes_Patients,sector lifescience healthcaredata category hea...,"{'sector': 'lifescience healthcaredata', 'cate...",6531,2,6531 \n Diabetes_Patients \n sector lifescienc...
3,3,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6530,3,6530 \n 21nov \n sector business servicesdata ...
4,4,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6529,4,6529 \n 21nov \n sector business servicesdata ...


In [10]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [11]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [12]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [13]:
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

TensorShape([1, 768])

In [14]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).numpy()[0]}
)

  0%|          | 0/3245 [00:00<?, ?ex/s]

In [15]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/4 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'field_solutiondescription', 'Key-Words', 'nid', '__index_level_0__', 'text', 'embeddings'],
    num_rows: 3245
})

In [23]:
question = "powerbi"
# question = "how to build conda"
# question = "agency problem"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [24]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [25]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [29]:
samples_df

,title,field_solutiondescription,Key-Words,nid,__index_level_0__,text,embeddings,scores
4,Tool to Convert MSTR Reports to Power BI,supports multiple data sources convert type ms...,"['supports', 'multiple', 'data', 'sources', 'c...",1276,2895,1276 \n Tool to Convert MSTR Reports to Power ...,"[-0.1245412677526474, 0.15979784727096558, -0....",43.902161
3,Power supply Prediction,daily forecast power load next 24 hours 15 min...,"['daily', 'forecast', 'power', 'load', 'next',...",527,3139,527 \n Power supply Prediction \n daily foreca...,"[-0.207869291305542, -0.38544952869415283, -0....",41.127373
2,Credit Risk Analysis PowerBi,solution leverages models created using analyt...,"['solution', 'leverages', 'models', 'created',...",3483,1605,3483 \n Credit Risk Analysis PowerBi \n soluti...,"[-0.28095489740371704, -0.021361008286476135, ...",40.487026
1,Medical Data Analysis Dashboard,aws medical data analysis using powerbi,"['aws', 'medical', 'data', 'analysis', 'using'...",6382,131,6382 \n Medical Data Analysis Dashboard \n aws...,"[-0.13937857747077942, -0.36679184436798096, -...",34.635521
0,Medical Data Analysis Using PowerBI,medical data analysis using powerbi,"['medical', 'data', 'analysis', 'using', 'powe...",6489,43,6489 \n Medical Data Analysis Using PowerBI \n...,"[-0.18395045399665833, -0.3286629915237427, -0...",30.488407


In [26]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.field_solutiondescription}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    # print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

COMMENT: supports multiple data sources convert type mstr report equivalent table power bi require executed mstr environment platform dependencies ui creation using electron js rich ui easy deployment saas loose coupling ui executor owning service line cloud data platform value proposition 1 reduce manual efforts errors maintain consistency report transition experience classification solutions accelerator technology used python powerbi efforts saving reduce 15 20 conversion time solution version 1 solution champion kaloya sarita sarita kaloya capgemini com sridhara sandeep kumar sandeep kumar sridhara capgemini com demo availability yes code check licencese cost involved yes click enquire team
SCORE: 43.90216064453125
TITLE: Tool to Convert MSTR Reports to Power BI

COMMENT: daily forecast power load next 24 hours 15 minutes time intervals determined using best statistical model least error owning service line industrial markets value proposition 1 forecast power need advance reduce co

In [ ]:
#working 1 december 

To build a semantic search engine for a dataset with three columns (title, field_solutiondescription, nid), one could use a pre-trained language model to extract features from the concatenated text in the columns. The features can then be fed into a nearest neighbors algorithm to find records that are most similar to a query. This can be done using the following steps:

Import the required libraries, such as pandas, scikit-learn, and transformers.
Load the dataset into a pandas dataframe, and concatenate the text in the three columns into a single field.
Use a TfidfVectorizer to extract features from the concatenated field.
Use a NearestNeighbors algorithm to find the records that are most similar to a query.
Return the results of the nearest neighbors search in a user-friendly format, such as a pandas dataframe or a list of dictionaries.
Alternatively, one could use a deep learning model, such as BERT, to generate embeddings for the text in the dataset and use a similarity measure, such as cosine similarity, to find records that are most similar to a query.

The code above is a simple implementation of a semantic search engine. A semantic search engine attempts to understand the user's query and match it to relevant documents in a dataset.

The code first extracts the 'title' and 'field_solutiondescription' columns from the input dataframe df and concatenates them into a single column called 'data_concat'. It then removes any rows in df that contain NaN values and resets the indices.

Next, the code uses a TfidfVectorizer to extract features from the 'data_concat' column. A TfidfVectorizer is a tool used in natural language processing to convert text into numerical vectors that can be used in machine learning algorithms. It uses the tf-idf weighting scheme to assign importance to each word in the text, with words that appear frequently in the document but not across the entire dataset being given a higher weight.

The code then uses a NearestNeighbors algorithm to find the records in df that are most similar to a given query. NearestNeighbors is a type of instance-based learning algorithm that identifies the closest matches to a query based on the feature vectors generated by the TfidfVectorizer.

Finally, the code defines a function semantic_search() that takes a query as input and returns the most similar records from df based on the results of the NearestNeighbors algorithm. The function first converts the query into a numerical vector using the TfidfVectorizer, then uses the search object to find the nearest neighbors to the query in the index of vectors generated earlier. It then returns the relevant rows from df.





In [76]:
import pandas as pd

import pyarrow as pa
import datasets

import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
df = df[['title', 'field_solutiondescription','Key-Words','nid']]
df.head()

,title,field_solutiondescription,Key-Words,nid
0,$finance,sector financial servicesdata category financi...,"{'sector': 'financial servicesdata', 'category...",6533
1,House_price_prediction_final,house_price_prediction_final,['house_price_prediction_final'],6532
2,Diabetes_Patients,sector lifescience healthcaredata category hea...,"{'sector': 'lifescience healthcaredata', 'cate...",6531
3,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6530
4,21nov,sector business servicesdata category adaption...,"{'sector': 'business servicesdata', 'category'...",6529


In [77]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

 
# extract the text and description fields from the data frame
text = df['title']
description = df['field_solutiondescription']

# concatenate the text and description fields into a single field
df['data_concat'] = text + ' ' + description

 
# remove rows that contain NaN values and reset the indices
df = df.dropna().reset_index(drop=True)

# use a TfidfVectorizer to extract features from the concatenated field
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(df['data_concat'])

# use a NearestNeighbors algorithm to find the records that are most similar to a query
search = NearestNeighbors(n_neighbors=100)
search.fit(features)

# define a function that takes a query as input and returns the most similar records from the data frame
def semantic_search(query):
    query_features = vectorizer.transform([query])
    distances, indices = search.kneighbors(query_features)
    return df.loc[indices[0], :]

In [91]:
# define the query
query = 'machine learning classification'

# search for similar records in the data frame
results = semantic_search(query)

# save the results as a CSV file
results.to_csv('results.csv')

results.head()

,title,field_solutiondescription,Key-Words,nid,data_concat
3130,Operation Monitoring,operation monitoring solution gains real time ...,"['operation', 'monitoring', 'solution', 'gains...",248,Operation Monitoring operation monitoring solu...
2175,Learning and Development,dashboard provides insights learning developme...,"['dashboard', 'provides', 'insights', 'learnin...",2231,Learning and Development dashboard provides in...
2329,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1965,IrisExp machine learning model file trained re...
2328,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1966,IrisExp machine learning model file trained re...
2319,logreg,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1995,logreg machine learning model file trained rec...


In [92]:
print(results.shape)


(100, 5)


In [93]:
from IPython.display import display

# load the CSV file into a Pandas data frame
df22 = pd.read_csv('results.csv')

# display the full data frame
display(df22)


,Unnamed: 0,title,field_solutiondescription,Key-Words,nid,data_concat
0,3130,Operation Monitoring,operation monitoring solution gains real time ...,"['operation', 'monitoring', 'solution', 'gains...",248,Operation Monitoring operation monitoring solu...
1,2175,Learning and Development,dashboard provides insights learning developme...,"['dashboard', 'provides', 'insights', 'learnin...",2231,Learning and Development dashboard provides in...
2,2329,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1965,IrisExp machine learning model file trained re...
3,2328,IrisExp,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1966,IrisExp machine learning model file trained re...
4,2319,logreg,machine learning model file trained recognize ...,"['machine', 'learning', 'model', 'file', 'trai...",1995,logreg machine learning model file trained rec...
...,...,...,...,...,...,...
95,2483,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",1728,2018 Historical Hourly Weather Data for 50 U.S...
96,2385,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",1861,2018 Historical Hourly Weather Data for 50 U.S...
97,2134,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",2319,2018 Historical Hourly Weather Data for 50 U.S...
98,1868,2018 Historical Hourly Weather Data for 50 U.S...,complimentary sample past historical hourly we...,"['complimentary', 'sample', 'past', 'historica...",3067,2018 Historical Hourly Weather Data for 50 U.S...


In [94]:
# !pip install gradio

import gradio as gr

# define the input and output components of the app
# define the input and output components of the app
# define the input and output components of the app
inputs = gr.inputs.Textbox(label="Enter a query:")
outputs = gr.outputs.Dataframe(label="Results", type="pandas", max_rows=100, max_cols=10)


# define the app's function
def semantic_search_app(query):
    results = semantic_search(query)
    return results

# launch the app
gr.Interface(
    semantic_search_app,
    inputs=inputs,
    outputs=outputs,
    title="Semantic Search",
    description="Search for similar records in a dataset using natural language processing"
).launch()


/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:127: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

*****using faiss cpu - 1 decemeber **bold text**

To build a semantic search engine using the faiss library, you can follow these steps:

Install the faiss library by running pip install faiss

Import the necessary modules from the faiss library, including faiss.IndexFlat, which is the main indexing class used for building a search engine.

Load the data into a Pandas dataframe, and select the columns you want to use for building the search engine. In this case, the columns are title, field_solutiondescription, Key-Words, and nid.

Convert the dataframe to a NumPy array using the .values attribute.

Create an instance of the faiss.IndexFlat class, and specify the dimension of the data (which is equal to the number of columns in the dataframe).

Add the rows of the dataframe to the index using the .add() method.

To perform a search, you can use the .search() method, which takes a query vector as input and returns a list of indices of the most similar records in the index.

You can then use these indices to retrieve the corresponding records from the dataframe and display the results.

This is just a high-level overview of the steps involved in building a semantic search engine using the faiss library. You may need to do some preprocessing of the data, such as converting text fields to numerical vectors using a vectorizer, and you may also want to experiment with different indexing algorithms and hyperparameters to improve the performance of the search engine.

In [ ]:
##25 november logic

In [ ]:
# https://www.kaggle.com/code/ajitrajput/semantic-search-engine-using-nlp/notebook

In [ ]:
import pandas as pd
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")
df1 = df[['title', 'field_solutiondescriptionimage','Key-Words','nid']]
df1.head()

,title,field_solutiondescriptionimage,Key-Words,nid
0,$finance,/sites/default/files/img_35.jpg,"{'sector': 'financial servicesdata', 'category...",6533
1,House_price_prediction_final,/sites/default/files/img_74.jpg,['house_price_prediction_final'],6532
2,Diabetes_Patients,/sites/default/files/img_80.jpg,"{'sector': 'lifescience healthcaredata', 'cate...",6531
3,21nov,/sites/default/files/img_85.jpg,"{'sector': 'business servicesdata', 'category'...",6530
4,21nov,/sites/default/files/img_58.jpg,"{'sector': 'business servicesdata', 'category'...",6529


In [95]:
def search_for_Keyword(dataframe, search_string):
    search=search_string.lower().split()
    for i in search:
        if i in list(stopwords.words('english')):
            search.remove(i)
    #print(search)
    lengths = {}
    indices_dict = {}
    for j in range(len(search)):
        indices = []
        for i in range(len(df)):
            if (search[j] in df['title'][i]) or (search[j] in df['Key-Words'][i]):
                indices.append(i)
        indices_dict[search[j]] = indices
        lengths[search[j]] = len(indices)
    
    keyword = [i for i in lengths if lengths[i]==max(lengths.values())]
    
    print("The complete items found are " + str(max(lengths.values())) + " for "+keyword[0]+". The top " +"paths are: \n")
    for i in indices_dict[keyword[0]]:  #top n values to display
        print(df["title"][i])
        print(df["field_solutiondescription"][i])
        print(df["field_solutiondescriptionimage"][i])
        print(df["nid"][i])
        print('\n\n')
        
    """    
    for i in range(len(dataframe)):
        if (keyword in df['title'][i]) or (keyword in df['Key-Words'][i]):
            indices.append(i)
    length1 = len(indices)
    #print(length1)
    #for i in range(5):
    #print(df["field_solutiondescriptionimage"][indices[i]])
    
    if length1 >= 5:
        print("The complete items found are " + str(length1) + ". The top "+"5 "+"paths are: \n")
        for i in range(5):
            print(df["field_solutiondescriptionimage"][indices[i]])
    elif (len(indices) >= 1) and (len(indices) < 5):
        print("The complete items found are " + str(length1) + ". The top "+str(length1)+" paths are: \n")
        for i in range(length1):
              print(df["field_solutiondescriptionimage"][indices[i]])
    else:
         print("There is no content against your search!!, Try again with your search!!!")"""

In [96]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [97]:
# df = pd.read_csv("CleanedDataKeyWords.csv")

# searchStr = input("Enter search word: ") #financial data #Financial data information is very crucial for the organization

searchStr = "diabetes"
df = pd.read_csv("/content/24.1_november_cleanedData (2).csv")

search_for_Keyword(df, searchStr)

The complete items found are 3 for diabetes. The top paths are: 

Pima Indian Diabetes Predictor
pima indian diabetes predictor
/sites/default/files/img_32.jpg
6348



diabetes_model
diabetes_model
/sites/default/files/img_7.jpg
5438



az-random-forest-classifier
diabetes_model
/sites/default/files/img_95.jpg
5363



